## Create Multi-Layer NN model using tensorflow

So I can adjust the parameters as I want...

In [61]:
import sys
sys.path.append("./src") # append to system path

from sklearn import cross_validation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.patches import Rectangle
style.use('ggplot')

In [62]:
def load_lcia_data(descs_p, target_p):
    X = pd.read_csv(descs_p,header=0,index_col=None)
    X = X.fillna(0)
    y = pd.read_csv(target_p,header=0,index_col=None)
    return X.values,y.values

def mre(true_y,pred_y):
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((true_y - pred_y) / true_y)) * 100

### Load Training data
The training data has 156 chemicals now. The rest 10 chemicals are test data
We also split the training and validation data here
We use smaller set (10%) to be the valdiation set (16 chemicals), as the limited size of training chemicals.

In [63]:
descs_p = '../data/descs/train/descs_Mar08_3839_train.csv'
target_p = '../data/target/train/GWP_train.csv'
X,y = load_lcia_data(descs_p, target_p)

trn_X, val_X, trn_y, val_y = cross_validation.train_test_split(
    X, y, test_size=0.1, random_state=42)

In [64]:
print trn_X.shape, trn_y.shape

(140, 3839) (140, 1)


### Load Testing data

In [65]:
descs_tst = '../data/descs/test/descs_Mar08_3839_test.csv'
target_tst = '../data/target/test/GWP_test.csv'
tst_X,tst_y = load_lcia_data(descs_tst, target_tst)

### Data Preprocessing
Normalization + PCA or Just Normalization

log scale of training target

In [66]:
trn_y,val_y,tst_y = np.log(trn_y),np.log(val_y),np.log(tst_y)

### Just Scaler

In [67]:
## Standard Scaler
this_scaler = StandardScaler()
trn_X = this_scaler.fit_transform(trn_X)
val_X = this_scaler.transform(val_X)
tst_X = this_scaler.transform(tst_X)

In [68]:
print trn_X.shape
print tst_X.shape
print np.mean(trn_X,0),np.std(trn_X,0)

(140, 3839)
(10, 3839)
[  8.94522551e-16  -4.85474756e-16  -3.96508223e-17 ...,   0.00000000e+00
  -6.97854473e-17   4.83740032e-17] [ 1.  1.  1. ...,  0.  1.  1.]


##  PCA

In [69]:
### PCA, don't run them together
# normalize the data first
pca = PCA(n_components = 60)

trn_X = pca.fit_transform(trn_X)
val_X = pca.transform(val_X)
tst_X = pca.transform(tst_X)

In [70]:
print trn_X.shape, tst_X.shape
print(reduce(lambda x,y:x+y,pca.explained_variance_ratio_))

(140, 60) (10, 60)
0.959570527671


In [71]:
plt.scatter(trn_X[:,0],trn_X[:,1])
plt.scatter(tst_X[:,0],tst_X[:,1])
plt.show()

In [72]:
n, bins, patches = plt.hist(trn_y, 50, normed=1, facecolor='green', alpha=0.75)
plt.show()

In [73]:
joblib.dump(this_scaler, '../nets/GWP/scaler.pkl') 
joblib.dump(pca, '../nets/GWP/pca.pkl') 

['../nets/GWP/pca.pkl']

### Build the model

In [87]:
def init_weights(shape):
    weights = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(weights)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

num_descs = trn_X.shape[1]
num_target = trn_y.shape[1]

print num_descs,num_target

60 1


In [88]:
##### 
##Define model structure

X = tf.placeholder(tf.float32,shape=[None,num_descs])
y = tf.placeholder(tf.float32,shape=[None,num_target])
tf.add_to_collection('X',X)
tf.add_to_collection('y',y)

#First layer
w1 = init_weights((num_descs,16)) 
b1 = bias_variable([16])
l1 = tf.add(tf.matmul(X,w1),b1)
l1 = tf.nn.relu(l1)

# Second layer
w2 = init_weights((16,16))
b2 = bias_variable([16])
l2 = tf.add(tf.matmul(l1,w2),b2)
l2 = tf.nn.relu(l2)

# Third layer
# w3 = init_weights((128,128))
# b3 = bias_variable([128])
# l3 = tf.add(tf.matmul(l2,w3),b3)
# l3 = tf.nn.sigmoid(l3)

#Output layer
w_out = init_weights((16,num_target))
b_out = bias_variable([num_target])
l_out = tf.matmul(l2,w_out) + b_out #no nonlinarity

pred = l_out
tf.add_to_collection('pred',pred)

In [89]:
#static parameters
BATCH_SIZE = 1
BETA = 0.01 #regularization weights

#Define loss and optimizer 
#Add regularization term
# regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w_out)
regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w_out)
cost = tf.reduce_mean(tf.square(pred - y) + BETA*regularizers)

#Gridient Descent Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)


# Initializing the variables
init = tf.global_variables_initializer()

## Training

In [86]:
%matplotlib auto
#Start Training
costs=[]

#save the model
saver = tf.train.Saver()

old_score = float('-inf')

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(3000):
        for i in range(0, len(trn_X),BATCH_SIZE):
            _, c = sess.run([optimizer,cost], feed_dict={X:trn_X[i:i+BATCH_SIZE], y:trn_y[i:i+BATCH_SIZE]})
        
        trn_score = r2_score(np.exp(trn_y),np.exp(sess.run(pred, feed_dict={X:trn_X, y:trn_y})))
        val_score = r2_score(np.exp(val_y),np.exp(sess.run(pred, feed_dict={X:val_X, y:val_y})))    
        val_mre = mre(np.exp(val_y),np.exp(sess.run(pred,feed_dict={X:val_X,y:val_y})))
        
        costs.append(val_score)
        if epoch % 10 == 0:
            print("Epoch = %d,Cost = %.2f,Training Accuracy = %.2f, Validation Accuracy = %.2f, Validation MRE =%.2f" % (epoch + 1,c,trn_score,val_score,val_mre))
  
    # final pred on the validation set
    final_pred_val = sess.run(pred,feed_dict={X:val_X})
    # prediction on the testing set
    final_pred_test = sess.run(pred,feed_dict={X:tst_X})
    
    for (y,y_hat) in zip(np.exp(tst_y),np.exp(final_pred_test)):
        print y,y_hat
    
    
    plt.plot(costs)
    plt.show()
    
    save_path = saver.save(sess, "../nets/GWP/GWP_Apr4.ckpt")
    saver.export_meta_graph("../nets/GWP/GWP_Apr4.meta")
    print("Model saved in file: %s" % save_path)

Using matplotlib backend: TkAgg
Epoch = 1,Cost = 2.35,Training Accuracy = -0.04, Validation Accuracy = -0.25, Validation MRE =66.82
Epoch = 11,Cost = 0.66,Training Accuracy = 0.81, Validation Accuracy = -1.32, Validation MRE =126.77
Epoch = 21,Cost = 0.28,Training Accuracy = 0.84, Validation Accuracy = -0.74, Validation MRE =105.71
Epoch = 31,Cost = 0.16,Training Accuracy = 0.97, Validation Accuracy = -0.10, Validation MRE =85.52
Epoch = 41,Cost = 0.10,Training Accuracy = 0.98, Validation Accuracy = -0.01, Validation MRE =74.05
Epoch = 51,Cost = 0.09,Training Accuracy = 0.97, Validation Accuracy = -0.55, Validation MRE =78.85
Epoch = 61,Cost = 0.09,Training Accuracy = 0.92, Validation Accuracy = -2.05, Validation MRE =88.64
Epoch = 71,Cost = 0.10,Training Accuracy = 0.93, Validation Accuracy = -2.60, Validation MRE =92.55
Epoch = 81,Cost = 0.09,Training Accuracy = 0.94, Validation Accuracy = -2.52, Validation MRE =91.26
Epoch = 91,Cost = 0.09,Training Accuracy = 0.97, Validation Accura

KeyboardInterrupt: 

In [59]:
tst_y = np.exp(tst_y)
final_pred_test = np.exp(final_pred_test)

In [60]:
MRE_this = mre(tst_y, final_pred_test)
R2_this = r2_score(tst_y, final_pred_test)
print R2_this
MRE_label = 'MRE: ' + str(round(MRE_this,2))

fig = plt.figure()
ax = fig.add_subplot(111)
est = plt.plot(tst_y, final_pred_test,'o', label='estimated values')

max_val = max(max(tst_y),max(final_pred_test))
plt.ylim([0,max_val+1])
plt.xlim([0,max_val+1])

thisLine = plt.plot(np.append(0,max_val+1), np.append(0,max_val+1), label='perfect prediction line')

plt.plot([],[],linewidth=0, label=MRE_label)
plt.legend(loc='upper left')
plt.show()

-0.107541670786
